In [ ]:
import gzip
import boto3
import pickle
import logging
import pandas as pd
from datetime import datetime
from azure.eventhub import EventHubConsumerClient

s3_client = boto3.client(
    "s3",
    aws_access_key_id='',
    aws_secret_access_key='',
)

In [ ]:
connection_str = 'Endpoint=sb://factored-datathon.servicebus.windows.net/;SharedAccessKeyName=datathon_group_1;SharedAccessKey=2GETvVt0FxyM0bo0Qau4inlmC/w3t4Uut+AEhAnAEgk=;EntityPath=factored_datathon_amazon_reviews_1'
consumer_group = 'team_datathon'
eventhub_name = 'factored_datathon_amazon_reviews_1'
client = EventHubConsumerClient.from_connection_string(connection_str, consumer_group, eventhub_name=eventhub_name)
logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.INFO)

def on_event(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))
    try:
        _ = s3_client.put_object(Body=gzip.compress(pickle.dumps(event.body_as_json())), Bucket='streaming-data-s3', Key=f'partition_{partition_context.partition_id}/event_{datetime.now().strftime("%H:%M:%S.%f")}.pkl.gzip')
    except Exception as e:
        logging.error(e)
    partition_context.update_checkpoint(event)
    
with client:
    client.receive(
        on_event=on_event,
        starting_position="-1",  # "-1" is from the beginning of the partition.
        # starting_position = '1714096'
    )
    # receive events from specified partition:
    # client.receive(on_event=on_event, partition_id='0')